In [48]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv('./static/dump/csv/result.csv')

# remove first two rows
df = df[2:]
head = df.head(0)

filtered_cols = [[f"{i}_questionaire_ans", f"{i}_questionaire_conf"] for i in [*range(1,16)]]
filtered_cols = [i for sublist in filtered_cols for i in sublist]

In [49]:
replacement_ans = {
    "Yes": 1,
    "Unsure": 0,
    "No": -1,    
}

replacement_conf = {
    "7-Extremely confident": 7,
    "6-Very confident": 6,
    "5-Somewhat confident": 5,
    "4-Neutral": 4,
    "3-Somewhat lacking confidence": 3,
    "2-Very lacking confidence": 2,
    "1-Extremely lacking confidence": 1,
}

condition = [
    "control",
    "honest summary",
    "honest chatbot",
    "misleading summary",
    "misleading chatbot"
]

# remove headers and replace values
df.replace(replacement_ans, inplace=True)
df.replace(replacement_conf, inplace=True)

result = df.copy()
result = result[['condition'] + [f"{i}_questionaire_ans" for i in [*range(1,16)]]]

for i in [*range(1,16)]:
    multiplier = 1 if i <= 10 else -1
    result[f"{i}_questionaire_ans"] *= multiplier
    
# result -> 1 = true memory / 0 = unsure / -1 = false memory

result

/var/folders/bw/64y3yr9n6v50fgm86wtx236c0000gn/T/ipykernel_33629/2364375147.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(replacement_ans, inplace=True)
/var/folders/bw/64y3yr9n6v50fgm86wtx236c0000gn/T/ipykernel_33629/2364375147.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(replacement_conf, inplace=True)


,condition,1_questionaire_ans,2_questionaire_ans,3_questionaire_ans,4_questionaire_ans,5_questionaire_ans,6_questionaire_ans,7_questionaire_ans,8_questionaire_ans,9_questionaire_ans,10_questionaire_ans,11_questionaire_ans,12_questionaire_ans,13_questionaire_ans,14_questionaire_ans,15_questionaire_ans
2,control,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1
3,honest summary,1,1,1,1,1,1,0,1,-1,0,0,1,0,-1,1
4,control,1,1,0,1,1,1,1,-1,1,1,-1,1,1,0,1
5,misleading summary,1,-1,1,-1,1,-1,-1,-1,1,1,1,1,1,1,1
6,honest summary,1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,1
7,honest chatbot,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1
8,misleading chatbot,1,1,1,-1,1,1,1,1,-1,1,0,0,-1,0,-1
9,misleading summary,1,1,1,1,0,1,1,-1,0,1,0,1,-1,-1,1
10,misleading chatbot,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,1,1,1,1,0
11,honest chatbot,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
def summarize(cat, df):
    # get sum
    df['overall'] = df[[f"{i}_questionaire_ans" for i in [*range(1,16)]]].sum(axis=1)
    df['induced'] = df[[f"{i}_questionaire_ans" for i in [*range(1,16)] if i > 10]].sum(axis=1)
    df['non induced'] = df[[f"{i}_questionaire_ans" for i in [*range(1,16) ] if i <= 10]].sum(axis=1)

    # get percentages in % in two decimals
    df['overall %'] = (df['overall'] / 24) * 100
    df['induced %'] = (df['induced'] / 12) * 100
    df['non induced %'] = (df['non induced'] / 12) * 100

    # filter
    df = df[['condition', 'overall', 'overall %', 'induced', 'induced %', 'non induced', 'non induced %']]

    print(cat)
    display(df.groupby('condition').mean().round(2))
    
true_memories = result.copy().replace(-1,0)
false_memories = result.copy().replace(1,0).replace(-1,1)
unsure_memories = result.copy().replace(1,-1).replace(0,1).replace(-1,0)
summarize('Non False Memories', true_memories)
summarize('Unsure Memories', unsure_memories)
summarize('False Memories', false_memories)

Non False Memories


,overall,overall %,induced,induced %,non induced,non induced %
condition,,,,,,
control,12.0,50.00,4.0,33.33,8.0,66.67
honest chatbot,6.5,27.08,2.0,16.67,4.5,37.50
honest summary,11.0,45.83,3.0,25.00,8.0,66.67
misleading chatbot,6.0,25.00,2.0,16.67,4.0,33.33
misleading summary,9.5,39.58,3.5,29.17,6.0,50.00


Unsure Memories


,overall,overall %,induced,induced %,non induced,non induced %
condition,,,,,,
control,2.0,8.33,0.5,4.17,1.5,12.50
honest chatbot,8.5,35.42,3.0,25.00,5.5,45.83
honest summary,2.0,8.33,1.0,8.33,1.0,8.33
misleading chatbot,2.5,10.42,2.0,16.67,0.5,4.17
misleading summary,1.5,6.25,0.5,4.17,1.0,8.33


False Memories


,overall,overall %,induced,induced %,non induced,non induced %
condition,,,,,,
control,1.0,4.17,0.5,4.17,0.5,4.17
honest chatbot,0.0,0.00,0.0,0.00,0.0,0.00
honest summary,2.0,8.33,1.0,8.33,1.0,8.33
misleading chatbot,6.5,27.08,1.0,8.33,5.5,45.83
misleading summary,4.0,16.67,1.0,8.33,3.0,25.00


In [53]:
def confidence(cat, result, raw):
    print(cat)
    # display(result)
    # display(raw[['condition', *[f"{i}_false_memories_conf" for i in [*range(1,25)]]]])
    
    # get confidence: for each row {i}_confidence = result[{i}_false_memories_ans] * raw[{i}_false_memories_conf]
    for i in [*range(1,16)]:
        result[f"{i}_conf"] = result[f"{i}_questionaire_ans"] * raw[f"{i}_questionaire_conf"]
    
    result['overall'] = result[[f"{i}_conf" for i in [*range(1,16)]]].sum(axis=1) / 15
    result['induced'] = result[[f"{i}_conf" for i in [*range(1,16)] if i if i >= 10]].sum(axis=1) / 5
    result['non induced'] = result[[f"{i}_conf" for i in [*range(1,16)] if i < 10]].sum(axis=1) / 10

    result = result[['condition', 'overall', 'induced', 'non induced']]
    display(result.groupby('condition').mean().round(2))
        
true_memories = result.copy().replace(-1,0)
false_memories = result.copy().replace(1,0).replace(-1,1)

confidence("Non False Memories", true_memories, df)
confidence("False Memories", false_memories, df)

Non False Memories


,overall,induced,non induced
condition,,,
control,4.30,5.1,3.9
honest chatbot,2.27,1.6,2.6
honest summary,4.63,4.1,4.9
misleading chatbot,2.57,2.9,2.4
misleading summary,3.57,4.9,2.9


False Memories


,overall,induced,non induced
condition,,,
control,0.23,0.5,0.10
honest chatbot,0.00,0.0,0.00
honest summary,0.77,1.1,0.60
misleading chatbot,2.30,1.7,2.60
misleading summary,1.33,1.1,1.45


In [59]:
score = pd.DataFrame()

# for each row in df, multiply answer by confidence
for i in [*range(1,16)]:
    # multiplier = -1 if i <= 10 else 1
    score['condition'] = df['condition']
    score[f"{i}_questionaire_score"] = multiplier * df[f"{i}_questionaire_ans"] * df[f"{i}_questionaire_conf"]
    score[f"{i}_questionaire_no"] = multiplier * df[f"{i}_questionaire_ans"]

# sum scores    
score['overall'] = score[[f"{i}_questionaire_score" for i in [*range(1,16)]]].sum(axis=1)
score['induced'] = score[[f"{i}_questionaire_no" for i in [*range(1,16)] if i > 10]].sum(axis=1)
score['weighted induced'] = score[[f"{i}_questionaire_score" for i in [*range(1,16)] if i > 10]].sum(axis=1)
score['non induced'] = score[[f"{i}_questionaire_no" for i in [*range(1,16)] if i <= 10]].sum(axis=1)
score['weighted non induced'] = score[[f"{i}_questionaire_score" for i in [*range(1,16)] if i <= 10]].sum(axis=1)

cond = ['condition', 'overall', 'induced', 'weighted induced', 'non induced', 'weighted non induced']
# score = score[cond + [f"{i}_false_memories_score" for i in [*range(1,25)]]]
score = score[cond]

# group by condition and average
print(f"score: 1 -> correct (true memory), 0 -> unsure, -1 if wrong (false memory)")
print("weight score: score {1,0,-1} * confidence [1,7] (how strong the memory is)")
print("higher score means less false memory")
score.groupby('condition').mean().round(2)


score: 1 -> correct (true memory), 0 -> unsure, -1 if wrong (false memory)
weight score: score {1,0,-1} * confidence [1,7] (how strong the memory is)
higher score means less false memory


,overall,induced,weighted induced,non induced,weighted non induced
condition,,,,,
control,27.0,-3.5,-17.0,7.5,44.0
honest chatbot,18.0,-2.0,-8.0,4.5,26.0
honest summary,35.0,-2.0,-11.5,7.0,46.5
misleading chatbot,-9.0,-1.0,-6.5,-1.5,-2.5
misleading summary,6.5,-2.5,-13.5,3.0,20.0
